# Part 2: Data Exploration & Insights

This notebook performs exploratory data analysis (EDA) and generates business insights to help the CRO understand win rate decline.

## Objectives
1. Explore the sales data structure and quality
2. Calculate custom metrics (RWWR, DFI)
3. Identify 3+ meaningful business insights
4. Explain each insight in plain business language with actionable recommendations

In [1]:
import sys
import os

# Add project root to path to import src modules
# Works whether run from project root or notebooks directory
if os.path.basename(os.getcwd()) == 'notebooks':
    project_root = os.path.dirname(os.getcwd())
else:
    project_root = os.getcwd()

if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Ensure output directories exist
os.makedirs(os.path.join(project_root, 'outputs', 'insights'), exist_ok=True)
os.makedirs(os.path.join(project_root, 'outputs', 'reports'), exist_ok=True)

import pandas as pd
import numpy as np
import matplotlibimport matplotlib.pyplot as plt
import seaborn as sns
from src.data_loader import load_sales_data, validate_data, add_derived_features
from src.metrics import (
    revenue_weighted_win_rate, deal_friction_index, win_rate, median_sales_cycle,
    win_rate_delta_by_segment, loss_concentration_ratio, sales_rep_win_rate_variance
)
from src.insights import generate_segment_insight, generate_lead_source_insight, generate_rep_performance_insight, format_insight
from src.utils import plot_win_rate_trend, create_summary_table

# Set visualization style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print(f"Project root: {project_root}")
print("Libraries imported successfully!")

Project root: /Users/shoaibmobassir/Desktop/SkyGeni1234
Libraries imported successfully!


## 1. Load and Validate Data

In [2]:
# Load data (use absolute path from project root)
data_path = os.path.join(project_root, 'data', 'skygeni_sales_data.csv')
df = load_sales_data(data_path)

# Validate data
is_valid, issues = validate_data(df)
print(f"Data validation: {'[OK] PASSED' if is_valid else '[ERROR] FAILED'}")
if issues:
    print("\nIssues found:")
    for issue in issues:
        print(f"  - {issue}")
else:
    print("No data quality issues detected.")

# Add derived features
df = add_derived_features(df)

print(f"\nDataset shape: {df.shape}")
print(f"Date range: {df['created_date'].min()} to {df['created_date'].max()}")
print(f"\nOutcome distribution:")
print(df['outcome'].value_counts())
print(f"\nOverall win rate: {(df['outcome'] == 'Won').sum() / len(df):.1%}")

Data validation: [OK] PASSED
No data quality issues detected.

Dataset shape: (5000, 17)
Date range: 2023-01-01 00:00:00 to 2024-03-26 00:00:00

Outcome distribution:
outcome
Lost    2737
Won     2263
Name: count, dtype: int64

Overall win rate: 45.3%


## 2. Custom Metrics: Revenue-Weighted Win Rate (RWWR) and Deal Friction Index (DFI)

In [3]:
# Calculate overall metrics
overall_wr = win_rate(df)
overall_rwwr = revenue_weighted_win_rate(df)
overall_dfi = deal_friction_index(df)

print("=== Overall Metrics ===")
print(f"Standard Win Rate: {overall_wr:.1%}")
print(f"Revenue-Weighted Win Rate (RWWR): {overall_rwwr:.1%}")
print(f"Deal Friction Index (DFI): {overall_dfi:.2f}")

print("\n=== Metric Interpretation ===")
print(f"RWWR vs Win Rate difference: {abs(overall_rwwr - overall_wr):.1%}")
if overall_rwwr < overall_wr:
    print("[WARN]  RWWR is LOWER than win rate → We're losing bigger deals!")
else:
    print("[OK] RWWR is higher than win rate → We're winning bigger deals")

print(f"\nDFI Interpretation:")
if overall_dfi > 1.2:
    print(f"[WARN]  DFI = {overall_dfi:.2f} → Lost deals take {overall_dfi:.1f}x longer than won deals")
    print("   This indicates qualification issues - reps are chasing dead deals")
elif overall_dfi < 0.8:
    print(f"[OK] DFI = {overall_dfi:.2f} → Lost deals close faster (good early disqualification)")
else:
    print(f"DFI = {overall_dfi:.2f} → Similar cycle lengths for won/lost deals")

=== Overall Metrics ===
Standard Win Rate: 45.3%
Revenue-Weighted Win Rate (RWWR): 46.1%
Deal Friction Index (DFI): 1.02

=== Metric Interpretation ===
RWWR vs Win Rate difference: 0.8%
[OK] RWWR is higher than win rate → We're winning bigger deals

DFI Interpretation:
DFI = 1.02 → Similar cycle lengths for won/lost deals


In [4]:
# Calculate RWWR by ACV bucket
rwwr_by_acv = {}
wr_by_acv = {}

for bucket in df['acv_bucket'].cat.categories:
    bucket_data = df[df['acv_bucket'] == bucket]
    if len(bucket_data) > 0:
        rwwr_by_acv[bucket] = revenue_weighted_win_rate(bucket_data)
        wr_by_acv[bucket] = win_rate(bucket_data)

comparison_df = pd.DataFrame({
    'Win Rate': wr_by_acv,
    'RWWR': rwwr_by_acv
})

comparison_df['Difference'] = comparison_df['RWWR'] - comparison_df['Win Rate']
comparison_df = comparison_df.sort_values('RWWR', ascending=False)

print("=== Win Rate vs RWWR by ACV Bucket ===")
print(comparison_df.to_string())
print("\nNOTE: Key Insight: RWWR reveals revenue impact, not just deal count impact")

=== Win Rate vs RWWR by ACV Bucket ===
                          Win Rate      RWWR  Difference
Large Enterprise (>$50k)  0.469290  0.468088   -0.001202
Mid-Market ($10k-$30k)    0.448587  0.454500    0.005913
SMB (<$10k)               0.448980  0.448686   -0.000294
Enterprise ($30k-$50k)    0.442708  0.444609    0.001900

NOTE: Key Insight: RWWR reveals revenue impact, not just deal count impact


## 3. Business Insight #1: Win Rate Decline by Segment

In [5]:
# Analyze win rate trends by ACV bucket
insight1 = generate_segment_insight(df, segment_col='acv_bucket', time_period_col='created_quarter')

print("=" * 80)
print("BUSINESS INSIGHT #1: Win Rate Decline by Segment")
print("=" * 80)
print(format_insight(insight1))
print("=" * 80)

# Visualize the trend
plot_win_rate_trend(df, segment_col='acv_bucket', time_col='created_quarter')
output_path = os.path.join(project_root, 'outputs', 'insights', 'win_rate_trend_by_acv.png')
plt.savefig(output_path, dpi=300, bbox_inches='tight')
plt.show()  # Close figure to free memory
print(f"[OK] Saved: {output_path}")

BUSINESS INSIGHT #1: Win Rate Decline by Segment
**What:** Win rate dropped most in acv_bucket='Enterprise ($30k-$50k)' segment

**Why it matters:** Focusing on this segment could have the biggest impact on overall win rate recovery

**Recommended action:** Review pricing, competition, and sales process for Enterprise ($30k-$50k) deals. Consider targeted enablement.


/Users/shoaibmobassir/Desktop/SkyGeni1234/src/utils.py:29: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  trend_data = df.groupby([time_col, segment_col]).agg({


[OK] Saved: /Users/shoaibmobassir/Desktop/SkyGeni1234/outputs/insights/win_rate_trend_by_acv.png


### Visualization: Win Rate Trend by ACV Bucket

![Win Rate Trend by ACV Bucket](outputs/insights/win_rate_trend_by_acv.png)

## 2.5. Additional Custom Metrics: WRΔ, LCR, SRWV

Beyond RWWR and DFI, we calculate three more metrics that answer "what changed?" and "where is the problem?":

1. **Win Rate Delta by Segment (WRΔ)**: Shows what changed over time
2. **Loss Concentration Ratio (LCR)**: Identifies 80/20 problem areas  
3. **Sales Rep Win Rate Variance (SRWV)**: Distinguishes process vs people problems

In [6]:
# Calculate Win Rate Delta by Segment (WRΔ)
# Compares win rate in last 2 quarters vs previous 2 quarters
print("=== Win Rate Delta by Segment (WRΔ) ===\n")
print("This metric answers: 'What changed?' - CROs don't ask 'what is the win rate', they ask 'what changed?'\n")

wr_delta_acv = win_rate_delta_by_segment(df, segment_col='acv_bucket', time_period_col='created_quarter')
if len(wr_delta_acv) > 0:
    print("Win Rate Change (Last 2Q vs Previous 2Q) by ACV Bucket:")
    for segment, delta in wr_delta_acv.items():
        direction = "↓" if delta < 0 else "↑"
        print(f"  {segment}: {direction} {abs(delta):.1%}")
    
    worst_segment = wr_delta_acv.idxmin()
    worst_delta = wr_delta_acv.min()
    print(f"\nNOTE: Worst performing segment: {worst_segment} (declined by {abs(worst_delta):.1%})")
else:
    print("Insufficient data for trend analysis")

# Calculate Loss Concentration Ratio (LCR)
print("\n=== Loss Concentration Ratio (LCR) ===\n")
print("This metric shows whether the problem is systemic or localized.\n")
print("High concentration = fix the few things causing most losses (80/20 principle)\n")

lcr = loss_concentration_ratio(df, segment_col='acv_bucket', top_n=3)
if lcr['concentration_ratio'] > 0:
    print(f"Top 3 segments account for {lcr['concentration_ratio']:.1%} of all losses")
    print(f"Top segments: {', '.join(lcr['top_segments'])}")
    print(f"\nNOTE: {'High concentration - Focus on fixing these few segments' if lcr['concentration_ratio'] > 0.5 else 'Moderate concentration - Problem is more distributed'}")
else:
    print("No loss data available")

# Calculate Sales Rep Win Rate Variance (SRWV)
print("\n=== Sales Rep Win Rate Variance (SRWV) ===\n")
print("This metric helps decide: coaching vs process fixes\n")

if 'sales_rep_id' in df.columns:
    srwv = sales_rep_win_rate_variance(df)
    if not pd.isna(srwv):
        print(f"SRWV: {srwv:.3f}")
        if srwv > 0.15:
            print("  → High variance (>0.15) = Process problem, not just people")
            print("  → Action: Fix process, not just coach individuals")
        elif srwv < 0.10:
            print("  → Low variance (<0.10) = Consistent process, individual coaching needed")
            print("  → Action: Focus on individual rep coaching")
        else:
            print("  → Moderate variance = Mix of process and individual factors")
            print("  → Action: Both process improvements and targeted coaching")
    else:
        print("Insufficient rep data for variance calculation")
else:
    print("Sales rep data not available")

=== Win Rate Delta by Segment (WRΔ) ===

This metric answers: 'What changed?' - CROs don't ask 'what is the win rate', they ask 'what changed?'

Win Rate Change (Last 2Q vs Previous 2Q) by ACV Bucket:
  SMB (<$10k): ↓ 1.3%
  Mid-Market ($10k-$30k): ↑ 4.2%
  Enterprise ($30k-$50k): ↓ 0.9%
  Large Enterprise (>$50k): ↓ 2.2%

NOTE: Worst performing segment: Large Enterprise (>$50k) (declined by 2.2%)

=== Loss Concentration Ratio (LCR) ===

This metric shows whether the problem is systemic or localized.

High concentration = fix the few things causing most losses (80/20 principle)

Top 3 segments account for 92.2% of all losses
Top segments: SMB (<$10k), Mid-Market ($10k-$30k), Large Enterprise (>$50k)

NOTE: High concentration - Focus on fixing these few segments

=== Sales Rep Win Rate Variance (SRWV) ===

This metric helps decide: coaching vs process fixes

SRWV: 0.025
  → Low variance (<0.10) = Consistent process, individual coaching needed
  → Action: Focus on individual rep coaching

/Users/shoaibmobassir/Desktop/SkyGeni1234/src/metrics.py:200: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  recent_wr = recent_data.groupby(segment_col).apply(
/Users/shoaibmobassir/Desktop/SkyGeni1234/src/metrics.py:200: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  recent_wr = recent_data.groupby(segment_col).apply(
/Users/shoaibmobassir/Desktop/SkyGeni1234/src/metrics.py:204: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=Fals

In [7]:
# Detailed analysis by ACV bucket
summary_by_acv = create_summary_table(df, 'acv_bucket')
print("\n=== Detailed Metrics by ACV Bucket ===")
print(summary_by_acv.to_string())

# Compare recent vs earlier periods
recent_quarters = df['created_quarter'].unique()[-2:]
earlier_quarters = df['created_quarter'].unique()[:-2]

recent_data = df[df['created_quarter'].isin(recent_quarters)]
earlier_data = df[df['created_quarter'].isin(earlier_quarters)]

print(f"\n=== Recent Periods ({recent_quarters[0]} to {recent_quarters[-1]}) ===")
recent_summary = create_summary_table(recent_data, 'acv_bucket')
print(recent_summary[['deal_count', 'win_rate', 'rwwr']].to_string())

print(f"\n=== Earlier Periods ({earlier_quarters[0]} to {earlier_quarters[-1]}) ===")
earlier_summary = create_summary_table(earlier_data, 'acv_bucket')
print(earlier_summary[['deal_count', 'win_rate', 'rwwr']].to_string())


=== Detailed Metrics by ACV Bucket ===
                          deal_count  total_acv       avg_acv  median_cycle  win_rate      rwwr       dfi
acv_bucket                                                                                               
Large Enterprise (>$50k)        1042   78453075  75290.858925          64.5  0.469290  0.468088  1.015625
Mid-Market ($10k-$30k)          1663   27081193  16284.541792          64.0  0.448587  0.454500  1.056911
Enterprise ($30k-$50k)           384   15264378  39750.984375          66.5  0.442708  0.444609  0.956204
SMB (<$10k)                     1911   10633818   5564.530612          64.0  0.448980  0.448686  0.984375

=== Recent Periods (2024Q1 to 2023Q2) ===
                          deal_count  win_rate      rwwr
acv_bucket                                              
Large Enterprise (>$50k)         426  0.492958  0.492397
Mid-Market ($10k-$30k)           653  0.427259  0.435409
Enterprise ($30k-$50k)           146  0.431507  0.430

/Users/shoaibmobassir/Desktop/SkyGeni1234/src/utils.py:119: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  summary = df.groupby(groupby_col).agg({
/Users/shoaibmobassir/Desktop/SkyGeni1234/src/utils.py:119: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  summary = df.groupby(groupby_col).agg({
/Users/shoaibmobassir/Desktop/SkyGeni1234/src/utils.py:119: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  summary = df.groupby(group

## 4. Business Insight #2: Lead Source Quality Impact

In [8]:
insight2 = generate_lead_source_insight(df)

print("=" * 80)
print("BUSINESS INSIGHT #2: Lead Source Quality Impact")
print("=" * 80)
print(format_insight(insight2))
print("=" * 80)

# Detailed lead source analysis
source_metrics = df.groupby('lead_source').agg({
    'outcome': lambda x: (x == 'Won').sum() / len(x),
    'sales_cycle_days': 'median',
    'deal_amount': ['mean', 'sum'],
    'deal_id': 'count'
}).round(2)

source_metrics.columns = ['win_rate', 'median_cycle', 'avg_acv', 'total_acv', 'deal_count']

# Add RWWR
for source in df['lead_source'].unique():
    source_data = df[df['lead_source'] == source]
    source_metrics.loc[source, 'rwwr'] = revenue_weighted_win_rate(source_data)

source_metrics = source_metrics.sort_values('win_rate', ascending=True)
print("\n=== Lead Source Performance Summary ===")
print(source_metrics.to_string())

BUSINESS INSIGHT #2: Lead Source Quality Impact
**What:** Deals from 'Outbound' source have lower win rate (45.5%) and longer cycles (66 days)

**Why it matters:** Marketing spend on Outbound is inflating pipeline volume without quality. This wastes sales time and resources.

**Recommended action:** Rebalance marketing spend toward higher-intent sources. Tighten MQL→SQL qualification criteria for Outbound leads.

=== Lead Source Performance Summary ===
             win_rate  median_cycle   avg_acv  total_acv  deal_count      rwwr
lead_source                                                                   
Partner          0.44          61.0  25867.81   32076082        1240  0.440501
Inbound          0.46          64.0  27041.87   34126837        1262  0.472259
Outbound         0.46          66.0  25963.35   32350340        1246  0.435624
Referral         0.46          65.0  26261.35   32879205        1252  0.494247


In [9]:
# Visualize lead source performance
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Win rate by source
source_metrics_sorted = source_metrics.sort_values('win_rate', ascending=True)
axes[0].barh(source_metrics_sorted.index, source_metrics_sorted['win_rate'], color='steelblue')
axes[0].set_xlabel('Win Rate', fontsize=12)
axes[0].set_title('Win Rate by Lead Source', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='x')

# Median cycle by source
source_metrics_cycle = source_metrics.sort_values('median_cycle', ascending=True)
axes[1].barh(source_metrics_cycle.index, source_metrics_cycle['median_cycle'], color='coral')
axes[1].set_xlabel('Median Sales Cycle (Days)', fontsize=12)
axes[1].set_title('Sales Cycle Length by Lead Source', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
output_path = os.path.join(project_root, 'outputs', 'insights', 'lead_source_analysis.png')
plt.savefig(output_path, dpi=300, bbox_inches='tight')
plt.show()
print(f"[OK] Saved: {output_path}")

[OK] Saved: /Users/shoaibmobassir/Desktop/SkyGeni1234/outputs/insights/lead_source_analysis.png

### Visualization: Lead Source Analysis

![Lead Source Analysis](outputs/insights/lead_source_analysis.png)

## 5. Business Insight #3: Rep-Level Performance Patterns

In [10]:
insight3 = generate_rep_performance_insight(df)

print("=" * 80)
print("BUSINESS INSIGHT #3: Rep-Level Performance Patterns")
print("=" * 80)
print(format_insight(insight3))
print("=" * 80)

# Rep performance analysis
rep_metrics = df.groupby('sales_rep_id').agg({
    'outcome': lambda x: (x == 'Won').sum() / len(x),
    'sales_cycle_days': 'median',
    'deal_id': 'count',
    'deal_amount': 'sum'
}).rename(columns={
    'outcome': 'win_rate',
    'sales_cycle_days': 'median_cycle',
    'deal_id': 'deal_count',
    'deal_amount': 'total_acv'
})

# Calculate DFI by rep
for rep in df['sales_rep_id'].unique():
    rep_data = df[df['sales_rep_id'] == rep]
    dfi = deal_friction_index(rep_data)
    rep_metrics.loc[rep, 'dfi'] = dfi if not pd.isna(dfi) else np.nan

rep_metrics = rep_metrics.sort_values('deal_count', ascending=False)
print("\n=== Top 10 Reps by Deal Volume ===")
print(rep_metrics.head(10).to_string())


BUSINESS INSIGHT #3: Rep-Level Performance Patterns
**What:** Rep rep_2 has normal deal volume (221 deals) but high Deal Friction Index (1.42)

**Why it matters:** Activity looks fine, but effectiveness is not. This rep is spending too much time on deals that won't close, indicating qualification issues.

**Recommended action:** Provide coaching to rep_2 on deal qualification and exit discipline. Review their qualification criteria and early-stage discovery process.

=== Top 10 Reps by Deal Volume ===
              win_rate  median_cycle  deal_count  total_acv       dfi
sales_rep_id                                                         
rep_20        0.469565          61.0         230    6250914  1.090909
rep_16        0.470852          65.0         223    5696674  0.817568
rep_2         0.461538          59.0         221    6293464  1.420000
rep_3         0.472222          63.0         216    5236648  0.816901
rep_22        0.400943          65.0         212    4633921  0.914286
re

In [11]:
# Visualize rep performance: Volume vs Effectiveness
fig, ax = plt.subplots(figsize=(12, 8))

# Scatter plot: Deal count vs DFI
scatter = ax.scatter(rep_metrics['deal_count'], rep_metrics['dfi'], 
                     s=rep_metrics['win_rate']*500, 
                     c=rep_metrics['win_rate'], 
                     cmap='RdYlGn', alpha=0.6, edgecolors='black')

ax.set_xlabel('Deal Count (Volume)', fontsize=12)
ax.set_ylabel('Deal Friction Index (DFI)', fontsize=12)
ax.set_title('Rep Performance: Volume vs Effectiveness\n(Size = Win Rate, Color = Win Rate)', 
             fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)

# Add reference lines
ax.axhline(y=1.0, color='gray', linestyle='--', alpha=0.5, label='DFI = 1.0 (Baseline)')
ax.axhline(y=1.2, color='red', linestyle='--', alpha=0.5, label='DFI = 1.2 (Warning)')

# Annotate problematic reps
high_dfi = rep_metrics[rep_metrics['dfi'] > 1.2]
for idx, row in high_dfi.head(5).iterrows():
    ax.annotate(idx, (row['deal_count'], row['dfi']), 
               fontsize=8, alpha=0.7)

plt.colorbar(scatter, label='Win Rate')
plt.legend()
plt.tight_layout()
output_path = os.path.join(project_root, 'outputs', 'insights', 'rep_performance.png')
plt.savefig(output_path, dpi=300, bbox_inches='tight')
plt.show()
print(f"[OK] Saved: {output_path}")

[OK] Saved: /Users/shoaibmobassir/Desktop/SkyGeni1234/outputs/insights/rep_performance.png


### Visualization: Rep Performance Analysis

![Rep Performance Analysis](outputs/insights/rep_performance.png)

## 6. Summary: Key Findings

### Custom Metrics Summary
1. **Revenue-Weighted Win Rate (RWWR)**: Reveals revenue impact, not just deal count
2. **Deal Friction Index (DFI)**: Identifies qualification issues (lost deals taking longer)

### Top 3 Business Insights
1. **Segment Decline**: Win rate dropped most in [segment] - explains revenue leakage
2. **Lead Source Quality**: [Source] generating low-quality pipeline - wastes sales time
3. **Rep Performance**: [Rep] has high volume but low effectiveness - needs coaching

### Recommended Actions
- Focus enablement on underperforming segments
- Rebalance marketing spend toward higher-quality sources
- Provide targeted coaching to reps with high DFI